In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
print(f"{torch.cuda.is_available()=}")
device="cpu"
if torch.cuda.is_available():
    device="cuda:0"
print(device)

In [ ]:
names_f = "tinyshakespeare/input.txt"
with open(names_f) as f:
    text = f.read()

#random.seed(42)
print(text[:30])
print(f"{len(text)=}")

chars = sorted(set(text))
voc_size = len(chars)
print(f"{chars[:100]=}")
print(f"{voc_size=}")

itos = dict()
stoi = dict()
for i, c in enumerate(chars):
    itos[i] = c
    stoi[c] = i
def encode(ss):
    return [stoi[c] for c in ss]
def decode(ii):
    return ''.join([itos[i] for i in ii])
print(encode("Hello\nWorld"))
print(decode(encode("Hello\nWorld")))

data = torch.tensor(encode(text), dtype=torch.long, device=device)
print(f"{data.shape=}")
print(data[:30])
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 256
head_size = 64
n_heads = 6
emb_size = head_size * n_heads
n_layers = 6
dropout_rate = 0.2
batch_size = 64
learning_rate = 1e-3
ctx = nullcontext() if device == 'cpu' else torch.amp.autocast(device_type="cuda", dtype=torch.float16)

In [ ]:
def get_batch(data, batch_size, device):
    ix = torch.randint(low=0,high=len(data)-block_size, size=(batch_size,), device=device)
    x = torch.stack([data[i : i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix]).to(device)
    return x, y

In [ ]:
class DotProductAttn(nn.Module):
    def __init__(self, is_mask) -> None:
        super().__init__()
        self.Q = nn.Linear(emb_size, head_size, bias=False)
        self.K = nn.Linear(emb_size, head_size, bias=False)
        self.V = nn.Linear(emb_size, head_size, bias=False)
        self.scale = head_size ** -0.5
        self.is_mask = is_mask
        if is_mask:
            self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout_rate)
    def forward(self, x: torch.Tensor):
        B,T,C = x.shape
        # x shape N,BL,EMB
        q = self.Q(x) # N,BL,H
        k = self.K(x) # N,BL,H
        v = self.V(x) # N,BL,H
        k = torch.transpose(k, -2, -1) # N,H,BL
        y = (q @ k) * self.scale # N,BL,BL
        if self.is_mask:
            y = torch.masked_fill(y, self.tril[:T, :T] == 0, float('-inf')) # Mask
        y = F.softmax(y, dim=-1)
        y = self.dropout(y)
        y = y @ v # N,BL,BL @ N,BL,H -> N,BL,H 
        return y

class MultiHeadAttn(nn.Module):
    def __init__(self, n_heads, is_mask) -> None:
        super().__init__()
        self.n_heads = n_heads
        self.heads = nn.ModuleList([DotProductAttn(is_mask) for _ in range(n_heads)])
        self.lin = nn.Linear(emb_size, emb_size, bias=True)
        self.dropout = nn.Dropout(dropout_rate)
    def forward(self, x):
        # x shape N,BL,EMB
        out = [head(x) for head in self.heads] #[(N,BL,H) x n_heads] 
        out = torch.cat(out, dim=-1) # N,BL,Hxn_heads
        out = self.lin(out)
        out = self.dropout(out)
        return out

class DecoderBlock(nn.Module):
    def __init__(self, n_heads) -> None:
        super().__init__()
        self.mh_attn = MultiHeadAttn(n_heads=n_heads, is_mask=True)
        self.FF = nn.Sequential(
            nn.Linear(emb_size, emb_size*4, bias=True),
            nn.ReLU(),
            nn.Linear(emb_size*4, emb_size, bias=True),
            nn.Dropout(dropout_rate)
        )
        self.ln1 = nn.LayerNorm(emb_size)
        self.ln2 = nn.LayerNorm(emb_size)
    def forward(self, x):
        # x shape N,BL,EMB
        y = x + self.mh_attn(x) # N,BL,Hxn_heads
        y = self.ln1(y)
        out = y + self.FF(y) # N,BL,Hxn_heads # FeedForward
        out = self.ln2(out)
        return out


class NanoGptModel(nn.Module):
    def __init__(self, voc_size) -> None:
        super().__init__()
        self.token_emb = nn.Embedding(voc_size, emb_size)
        self.position_emb = nn.Embedding(block_size, emb_size)
        self.blocks = nn.Sequential(*[DecoderBlock(n_heads) for _ in range(n_layers)])
        self.lin = nn.Linear(emb_size, voc_size, bias=True)
    def forward(self, ids):
        B, T = ids.shape
        token_out = self.token_emb(ids)
        pos_out = self.position_emb(torch.arange(T, device=device))
        out = token_out + pos_out
        out = self.blocks(out)
        out = self.lin(out)
        return out
    def calc_loss(self, logits, Y):
        logits = logits.transpose(1,2)
        return F.cross_entropy(logits, Y)
    @torch.no_grad()
    def generate(self, ids, max_new_tokens):
        for i in range(max_new_tokens):
            logits = self(ids[:,-block_size:])
            logits = logits[:,-1,:]
            prob = torch.softmax(logits, dim=-1)
            y = torch.multinomial(prob, num_samples=1)
            ids = torch.cat((ids, y), dim=-1)
        return ids.detach().cpu().numpy()
    
@torch.no_grad()
def estimate_loss(N):
    model.eval()
    WIN=[]
    for _ in range(N):
        x, y_target = get_batch(val_data, batch_size, device)
        with ctx:
            logits = model(x)
            loss = model.calc_loss(logits, y_target)
        WIN.append(loss.item())
    model.eval()
    return np.mean(WIN)

# head = DotProductAttn(is_mask=True)
# mha = MultiHeadAttn(n_heads=n_heads, is_mask=True)
# dec_block0 = DecoderBlock(n_heads)
# dec_block1 = DecoderBlock(n_heads)
model = NanoGptModel(voc_size=voc_size).to(device)
n_of_params = sum([p.numel() for p in model.parameters()])
print(f"Numel: {n_of_params:,}")

x, y = get_batch(train_data, batch_size, device)
out = model(x)
print(out.shape)
loss = model.calc_loss(out, y)
print("loss:", loss.item())

res = model.generate(x, 4)
print(res.shape)
print(decode(res[0]))

losst, lossv, lri = [],[],[]


In [ ]:
if device == "cpu":
    print("Exporting to ONNX...")
    torch.onnx.export(model, x, "gpt.onnx")
    print("Done")

In [ ]:
# Compilation
model.train()
#model = torch.compile(model)
model.train()
out = model(x)
print(out.shape)
loss = model.calc_loss(out, y)
print("loss:", loss.item())
loss.backward()

In [ ]:
# Training Loop
import math
warmup_iters = 100
lr_decay_iters = 5000
min_lr = 1e-4
learning_rate = 1e-3

def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
WIN = []
N = 1000
for i in range(N):
    if i > 0:
        lr = get_lr(i)
        lri.append(lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    x, y_target = get_batch(train_data, batch_size, device)
    logits=model(x)
    loss = model.calc_loss(logits, y_target)
    WIN.append(loss.item())
    if (i+1) % 10 == 0:
        avg_loss = np.mean(WIN)
        WIN=[]
        losst.append(avg_loss)
        loss_est = estimate_loss(10)
        lossv.append(loss_est)
        print(f"{i+1}: {losst[-1]=}, {lossv[-1]=}, {lr=}")

if losst and lossv:
    print(f"{losst[-1]=}, {lossv[-1]=}")

In [ ]:
if losst:
    plt.figure(figsize=(20,5))
    plt.grid()
    plt.plot(losst, color="red")
    plt.plot(lossv, color="blue")

In [ ]:
if lri:
    plt.figure(figsize=(20,5))
    plt.grid()
    plt.plot(lri)

In [ ]:
# Validation Loop
val_loss = estimate_loss(20)
print(f"{val_loss=}")

In [ ]:
x, y = get_batch(val_data, 1, device)
model.eval()
#x = torch.zeros((1,block_size), dtype=torch.long, device=device)
res = model.generate(x, 500)
print(decode(res[0,:block_size]))
print("--<START>--")
print(decode(res[0,block_size:]))

In [ ]:
torch.save(model.state_dict(), "gpt-state-dict.pt")